In [1]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.40rows/s]


,ontime,dep_delay,taxi_out,distance,is_eval_day
0,ontime,18.0,12.0,1334.0,False
1,ontime,0.0,11.0,1056.0,True
2,ontime,-3.0,14.0,199.0,True
3,late,17.0,11.0,1052.0,False
4,ontime,0.0,10.0,160.0,False


In [2]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_lm
OPTIONS(input_label_cols=['ontime'],
        model_type='logistic_reg',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1676.38query/s]                        


""


In [3]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL dsongcp.arr_delay_lm)

Downloading: 100%|██████████| 11/11 [00:01<00:00,  9.17rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,10,0.169790,0.167233,25.6,21657
1,0,9,0.171054,0.168219,25.6,21589
2,0,8,0.173285,0.170453,51.2,21052
3,0,7,0.190027,0.186490,25.6,21349
4,0,6,0.219350,0.214753,12.8,21277
5,0,5,0.261108,0.255400,6.4,21207
6,0,4,0.321923,0.316414,3.2,22013
7,0,3,0.415231,0.411193,1.6,22093
8,0,2,0.521449,0.519102,0.8,22753
9,0,1,0.606936,0.605817,0.4,21441


In [4]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_lm,
                 (
                     
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
    is_train_day = 'False'
                     
                 ),
                 STRUCT(0.7 AS threshold))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.08s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.964337,0.956535,0.935174,0.96042,0.167233,0.956284


In [5]:
%%bigquery
SELECT * FROM ML.WEIGHTS(MODEL dsongcp.arr_delay_lm)

Downloading: 100%|██████████| 4/4 [00:01<00:00,  3.47rows/s]


,processed_input,weight,category_weights
0,dep_delay,-0.132984,[]
1,taxi_out,-0.121715,[]
2,distance,0.000223,[]
3,__INTERCEPT__,4.762572,[]


In [6]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                        (
SELECT 12.0 AS dep_delay, 14.0 AS taxi_out, 1231 AS distance
                        ))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.23s/rows]


,predicted_ontime,predicted_ontime_probs,dep_delay,taxi_out,distance
0,ontime,"[{'label': 'ontime', 'prob': 0.850350772376713...",12.0,14.0,1231


In [7]:
%%bigquery
WITH predictions AS (
SELECT
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),
stats AS (
SELECT
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)
SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.17s/rows]


,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.836363,1301948,0.964337,283750,0.213091


In [8]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_lm
OPTIONS(input_label_cols=['ontime'],
        model_type='logistic_reg',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1796.28query/s]                        


""


In [9]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.23s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.967151,0.957706,0.938477,0.962405,0.165557,0.960865


In [10]:
%%bigquery
WITH predictions AS (
SELECT
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),
stats AS (
SELECT
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)
SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.28s/rows]


,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.84953,1299749,0.967151,283750,0.209839
